In [ ]:
%%capture
!pip install requests beautifulsoup4  sentence-transformers transformers
!pip install git+https://github.com/huggingface/accelerate
!pip install langchain chromadb langchain
!pip install langchain_chroma langchain_community

In [ ]:
import requests
from bs4 import BeautifulSoup

# Define your user agent string
headers = {'User-Agent': 'Your Name - your_email@example.com'}

def get_filings(ticker, filing_type="10-K"):
    url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={ticker}&type={filing_type}&dateb=&owner=exclude&count=100&output=xml"
    print(url)
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None
ticker="MSFT"
filings_xml = get_filings(ticker)

In [ ]:
import requests, os
from bs4 import BeautifulSoup,NavigableString


def parse_filings(filings_xml):
    """
    Gets the link from the "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=MSFT&type=10-K&dateb=&owner=exclude&count=100&output=xml"
    to Retrieve the target document from "https://www.sec.gov/Archives/edgar/data/789019/000095017023035122/0000950170-23-035122.txt"
    """
    soup = BeautifulSoup(filings_xml, 'xml')  # Parse as XML
    links = soup.find_all('filingHREF')
    # print(links)
    # print(links)
    for link in links:
        filing_page_url = link.text
        # print(filing_page_url)
        # Fetch the filing page content
        filing_page_content = requests.get(filing_page_url,headers=headers).text
        # print(filing_page_content)
        # Parse the filing page content
        filing_page_soup = BeautifulSoup(filing_page_content, 'html.parser')
        
        # Search for the target document link within the filing page
        document_link = filing_page_soup.find('a', href=lambda href: href and href.endswith('.txt'))
        # print(document_link)
        if document_link:
            return document_link.get('href')
    
    return None

document_url = parse_filings(filings_xml)
print(document_url)
exit()
c_document_url=document_url.replace("ix?doc=/","")
base_url ="https://www.sec.gov"
if document_url:
    print(f"Document URL: {base_url+c_document_url}")
else:
    print(f"Document {target_document} not found in filings XML")

html_content = requests.get(base_url+c_document_url, headers=headers).text

def get_text_from_innermost_tags(tag):
    if isinstance(tag, NavigableString):
        return tag.strip()
    else:
        # Check if this tag contains other tags
        if tag.find() is not None:
            # This tag contains other tags, so ignore it
            return ''
        else:
            # This tag does not contain other tags, so extract its text
            return tag.get_text(separator='\n', strip=True) +'\n'
            
soup = BeautifulSoup(html_content, 'html.parser')
start_tag = soup.find(lambda tag: tag.name == "span" and "UNITED STATES" in tag.get_text())
# print(start_tag)
if start_tag:
    # Extract the HTML from the starting tag onwards
    relevant_content = start_tag.get_text(separator='\n', strip= True)
    for element in start_tag.find_all_next():
        # print(element)
        relevant_content += get_text_from_innermost_tags(element) 
else:
    print("Starting point not found in the document")
    relevant_content = ""

# print(relevant_content)

/ix?doc=/Archives/edgar/data/789019/000095017023035122/msft-20230630.htm
Document URL: https://www.sec.gov/Archives/edgar/data/789019/000095017023035122/msft-20230630.htm


In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
with open('output.txt', 'w') as file:
    file.write(relevant_content)

loader = TextLoader('output.txt')
documents= loader.load()

text_splitter = CharacterTextSplitter(chunk_size =1000, chunk_overlap =200)
docs = text_splitter.split_documents(documents)

model_name = "all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_function = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
db = Chroma.from_documents(docs,embedding=embedding_function)

query = "what is the net Income for the year 2023"
docs= db.similarity_search(query)

print(docs[0].page_content)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


2023


2022


2021

Net income

$
72,361


$
72,738


$
61,271

Other comprehensive income (loss), net of tax:


Net change related to derivatives


(
14
)


6

19

Net change related to investments


(
1,444
)


(
5,360
)


(
2,266
)
Translation adjustments and other


(
207
)


(
1,146
)


873


Other comprehensive loss


(
1,665
)


(
6,500
)


(
1,374
)

Comprehensive income

$
70,696


$
66,238


$
59,897

Refer to accompanying notes.

59

PART II
Item 8

BALANCE
SHEETS

(In millions)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, accelerate

device ="cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)





Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
query= "What is the Debt ratio of this company?"
retriever = db.as_retriever(k=1)
relevant_chunks = retriever.invoke(query)
chunk_texts = [doc.page_content for doc in relevant_chunks]
combined_input = "Userquery :" + query + "\n\n" + "\n Context:".join(chunk_texts)

input_ids = tokenizer(combined_input, return_tensors="pt").input_ids.to(device)

output = model.generate(input_ids, max_new_tokens=200, num_return_sequences=1)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Userquery :What is the Debt ratio of this company?

Amount


Weighted
Average Life

Year Ended June 30,

2023

2022

Technology-based

$
522

7
years


$
2,611

4
years

Customer-related


0

0
years

2,837

9
years

Marketing-related


7

5
years

233

4
years

Contract-based


12

3
years

0

0
years


Total

$
541

6
years


$

5,681

7
years

Intangible assets amortization expense was $
2.5
billion, $
2.0
billion, and $
1.6
billion for fiscal years 2023, 2022, and 2021, respectively.

80

PART II
Item 8

The following table outlines the estimated future amortization expense related to intangible assets held as of June 30, 2023:


(In millions)


Year Ending June 30,


2024

$
2,363

2025


1,881

2026


1,381

2027


929

2028


652

Thereafter


2,160


Total

$
9,366

NOTE 11 — DEBT

The components of debt were as follows:

(In millions, issuance by calendar year)

Maturities
(calendar year)

Stated Interest
Rate


Effective Interest
Rate

June 30,
2023


June 30,
2022


2009
iss

In [ ]:
import torch
import gc

# Assume you have some model and data loaded
# model = ...
# inputs = ...

# Perform operations
# output = model(inputs)

# Now clear GPU memory

gc.collect()  # run garbage collection
torch.cuda.empty_cache()  # clear the PyTorch cache



In [ ]:
!nvidia-smi

Sat Jun  8 23:11:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              35W /  70W |   7439MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import re

def detect_and_convert_to_markdown(text):
    lines = text.strip().split("\n")
    rows = []
    current_row = []
    max_columns = 0

    # Helper function to add row if it's complete
    def add_row_if_complete():
        nonlocal max_columns
        if current_row:
            rows.append(current_row[:])
            if len(current_row) > max_columns:
                max_columns = len(current_row)
            current_row.clear()

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Split the line by spaces, tabs, or other delimiters to detect columns
        columns = re.split(r'\s{2,}|\t', line)
        if len(columns) > 1:
            add_row_if_complete()
            current_row.extend(columns)
            add_row_if_complete()
        else:
            current_row.append(line)
            add_row_if_complete()

    # Handle the last row if any
    add_row_if_complete()

    # Construct the markdown table
    if rows:
        headers = ["Column " + str(i + 1) for i in range(max_columns)]
        markdown_table = f"| {' | '.join(headers)} |\n"
        markdown_table += f"|{'|'.join(['-' * (len(header) + 2) for header in headers])}|\n"

        for row in rows:
            markdown_table += f"| {' | '.join(row + [''] * (max_columns - len(row)))} |\n"

        return markdown_table
    else:
        return "No table detected in the text."

# Sample input text
text = generated_text

# Generate the markdown table
markdown_table = detect_and_convert_to_markdown(text)
print(markdown_table)

# Optionally, save to a markdown file
with open("output_table.md", "w") as file:
    file.write(markdown_table)



| Column 1 |
|----------|
| Userquery :What is the Debt ratio of this company? |
| Amount |
| Weighted |
| Average Life |
| Year Ended June 30, |
| 2023 |
| 2022 |
| Technology-based |
| $ |
| 522 |
| 7 |
| years |
| $ |
| 2,611 |
| 4 |
| years |
| Customer-related |
| 0 |
| 0 |
| years |
| 2,837 |
| 9 |
| years |
| Marketing-related |
| 7 |
| 5 |
| years |
| 233 |
| 4 |
| years |
| Contract-based |
| 12 |
| 3 |
| years |
| 0 |
| 0 |
| years |
| Total |
| $ |
| 541 |
| 6 |
| years |
| $ |
| 5,681 |
| 7 |
| years |
| Intangible assets amortization expense was $ |
| 2.5 |
| billion, $ |
| 2.0 |
| billion, and $ |
| 1.6 |
| billion for fiscal years 2023, 2022, and 2021, respectively. |
| 80 |
| PART II |
| Item 8 |
| The following table outlines the estimated future amortization expense related to intangible assets held as of June 30, 2023: |
| (In millions) |
| Year Ending June 30, |
| 2024 |
| $ |
| 2,363 |
| 2025 |
| 1,881 |
| 2026 |
| 1,381 |
| 2027 |
| 929 |
| 2028 |
| 652 |
| Therea